In [31]:
import base64
import datetime as dt
import hmac
import requests
import pandas as pd
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [32]:
# define the scope
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']

# add credentials to the account
creds = ServiceAccountCredentials.from_json_keyfile_name('../data/client_secret.json', scope)

# authorize the clientsheet 
client = gspread.authorize(creds)

In [34]:
# get the instance of the Spreadsheet
sheet = client.open('Carteira')

# get the first sheet of the Spreadsheet
sheet_instance = sheet.get_worksheet(0)

In [60]:
col = 26
row = 7

cripto_list = []

while(True):
    if not sheet_instance.cell(col=col,row=row).value:
        break

    cripto_list.append(
        { 
            "nome": sheet_instance.cell(col=col,row=row).value,
            "col": col,
            "row": row
        }
    )
    row += 1

In [61]:
cripto_list

[{'nome': 'BTC', 'col': 26, 'row': 7},
 {'nome': 'LINK', 'col': 26, 'row': 8},
 {'nome': 'AR', 'col': 26, 'row': 9},
 {'nome': 'MANA', 'col': 26, 'row': 10},
 {'nome': 'SAND', 'col': 26, 'row': 11},
 {'nome': 'UNI', 'col': 26, 'row': 12},
 {'nome': 'CRV', 'col': 26, 'row': 13},
 {'nome': 'SNX', 'col': 26, 'row': 14},
 {'nome': 'AAVE', 'col': 26, 'row': 15},
 {'nome': 'LDO', 'col': 26, 'row': 16},
 {'nome': 'GMX', 'col': 26, 'row': 17},
 {'nome': 'ETH', 'col': 26, 'row': 18},
 {'nome': 'ATOM', 'col': 26, 'row': 19},
 {'nome': 'ARB', 'col': 26, 'row': 20},
 {'nome': 'MATIC', 'col': 26, 'row': 21},
 {'nome': 'OP', 'col': 26, 'row': 22},
 {'nome': 'USDC', 'col': 26, 'row': 23}]

In [18]:
def send_signed_request(http_method, url_path, payload={}):
    def get_time():
        return dt.datetime.utcnow().isoformat()[:-3]+'Z'
    
    def signature(timestamp, method, request_path, body, secret_key):
        if str(body) == '{}' or str(body) == 'None':
            body = ''
        message = str(timestamp) + str.upper(method) + request_path + str(body)
        mac = hmac.new(bytes(secret_key, encoding='utf8'), bytes(message, encoding='utf-8'), digestmod='sha256')
        d = mac.digest()
        return base64.b64encode(d)
    
    # set request header
    def get_header(request='GET', endpoint='', body:dict=dict()):
        cur_time = get_time()
        header = dict()
        header['CONTENT-TYPE'] = 'application/json'
        header['OK-ACCESS-KEY'] = os.environ.get('OKX_API_KEY')
        header['OK-ACCESS-SIGN'] = signature(cur_time, request, endpoint, body, os.environ.get('OKX_API_SECRET'))
        header['OK-ACCESS-TIMESTAMP'] = str(cur_time)
        header['OK-ACCESS-PASSPHRASE'] = os.environ.get('OKX_PASS_PHRASE')
        return header

    url = 'https://www.okx.com' + url_path
    header = get_header(http_method, url_path, payload)
    response = requests.get(url, headers=header)
    return response.json()

response = send_signed_request("GET", "/api/v5/account/balance", payload={})

In [19]:
for coin_details in response['data'][0]['details']:
    item = next(item for item in list_cripto if item["nome"] == coin_details['ccy'])
    sheet_instance.update_cell(item['row'], item['col']+1, coin_details['eqUsd'])